# 쇼핑 어시스턴트 구축하기

이 워크샵에서는 두 가지 다른 접근 방식으로 쇼핑 어시스턴트를 구축해보겠습니다.
이전에 다룬 tool 설계 방법과 context preloading 기법을 함께 활용해보겠습니다.

## 학습 목표
- 🏗️ **워크플로우 기반** 접근법으로 모듈형 어시스턴트 구축
- 🤖 **에이전트 기반** 접근법으로 통합형 어시스턴트 구축
- 📊 두 접근법의 장단점 비교 분석
- 🎯 개인화 기능으로 사용자 경험 향상

## 구축할 시스템 개요
- **라우터**: 사용자 의도에 따라 적절한 핸들러로 라우팅
- **주문 조회 핸들러**: DynamoDB에서 주문 내역 조회
- **상품 검색 핸들러**: OpenSearch로 상품 검색 및 추천
- **일반 문의 핸들러**: 정책, 환불 등 일반적인 고객 서비스

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
from config import BedrockModelId
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
from boto3.dynamodb.conditions import Key
from common_functions import converse_bedrock
import json
from pprint import pprint
import time

## 인프라 구성 요소

### 필요한 AWS 서비스
- **DynamoDB**: 사용자 정보, 주문 내역, 상품 리뷰 저장
- **OpenSearch Serverless**: 상품 카탈로그 검색
- **Bedrock**: LLM 모델 호스팅

### 데이터베이스 구조
- `UsersTable`: 사용자 프로필 및 개인화 정보
- `OrdersTable`: 주문 내역 및 배송 상태  
- `ReviewsTable`: 상품 리뷰 및 평점
- `products` 인덱스: 상품 카탈로그 (OpenSearch)

In [ ]:
def get_orders_with_user_id(user_id: str) -> dict:
    """
    Get the orders for a user.
    """
    table_name = "OrdersTable"
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table(table_name)
    # Query the table for orders with the given user_id
    try:
        response = table.query(
            IndexName='UserStatusIndex',
            KeyConditionExpression=Key('user_id').eq(int(user_id))
        )
        
        orders = response.get('Items', [])
        
        # Format orders for response
        formatted_orders = []
        for order in orders:
            formatted_order = {
                "order_id": order.get('order_id'),
                "timestamp": order.get('timestamp'),
                "item_id": order.get('item_id'),
                "delivery_status": order.get('delivery_status')
            }
            formatted_orders.append(formatted_order)
        
        return {
            "messageVersion": "1.0",
            "response": {
                "orders": formatted_orders
            }
        }
    except Exception as e:
        error_msg = f"Error getting orders: {str(e)}"
        return {
            "messageVersion": "1.0",
            "response": {
                "error": error_msg
            }
        }


def get_user_info(user_id: str) -> dict:
    """
    Get the address for a user.
    """

    table_name = "UsersTable"
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table(table_name)
    if not user_id:
        return "Error: user_id is required"
    
    try:
        response = table.get_item(
            Key={
                'id': user_id
            }
        )

        user = response.get('Item')

        if not user:
            return "Error: User not found"
        
        return "User info: " + str(user)

    except Exception as e:
        error_msg = f"Error getting user address: {str(e)}"
        print(error_msg)
        return "Error: " + error_msg

In [ ]:
dynamodb = boto3.resource('dynamodb', region_name='us-west-2')

def get_dynamodb_client():
    return dynamodb

def get_product_reviews(product_ids: list) -> dict:
    """
    Get reviews for multiple products using BatchGetItem.
    """
    product_ids = list(set(product_ids))
    try:
        # BatchGetItem can retrieve up to 100 items at once
        if len(product_ids) > 100:
            product_ids = product_ids[:100]  # Limit to first 100
            
        # Prepare the request items format for BatchGetItem
        request_items = {
            'ReviewsTable': {
                'Keys': [{'product_id': product_id} for product_id in product_ids]
            }
        }
        
        # Execute the BatchGetItem operation
        response = get_dynamodb_client().batch_get_item(RequestItems=request_items)
        
        # Process the results
        reviews_by_id = {}
        if 'Responses' in response and 'ReviewsTable' in response['Responses']:
            for item in response['Responses']['ReviewsTable']:
                product_id = item['product_id']
                reviews_by_id[product_id] = {
                    'avg_rating': item.get('avg_rating'),
                    'positive_keywords': item.get('positive_keywords'),
                    'negative_keywords': item.get('negative_keywords'),
                    'review_summary': item.get('review_summary')
                }
                
        return reviews_by_id
    except Exception as e:
        print(f"Error retrieving product reviews: {str(e)}")
        return {}

# 접근법 1: 워크플로우 기반 아키텍처

## 핵심 개념: 모듈화와 전문화

### 아키텍처 설계 철학
1. **단일 책임 원칙**: 각 핸들러는 하나의 기능에만 집중
2. **명시적 라우팅**: 사용자 의도를 명확히 분류하여 처리
3. **독립적 개발**: 각 모듈을 독립적으로 개발하고 테스트 가능

### 라우터 시스템
사용자의 메시지를 분석하여 적절한 전문 핸들러로 라우팅합니다:
- **1번**: 주문 조회 (Order History)
- **2번**: 상품 검색 (Product Search) 
- **3번**: 일반 문의 (General Inquiry)

In [ ]:
ROUTER_PROMPT = """# Enhanced AI Assistant Routing System

## Role Definition
You are an intelligent routing system designed to analyze user messages and direct them to the most appropriate specialized assistant. Your primary responsibility is to ensure users receive the most relevant and efficient assistance by matching their inquiries to the assistant best equipped to handle their specific needs.

## Available Assistants Overview

### Assistant 1: Order History Assistant
**Specialization**: Order tracking, purchase history
**Handles**: Past orders, delivery status

### Assistant 2: Product Search Assistant  
**Specialization**: Product discovery and search functionality
**Handles**: Finding products, search queries, category browsing, filtering

### Assistant 3: General Inquiry Assistant
**Specialization**: General customer service and miscellaneous inquiries
**Handles**: Policies, general questions, support issues, non-product related queries

## Routing Methodology

### Step 1: Message Analysis
- **Intent Recognition**: Identify the primary purpose of the user's message
- **Context Evaluation**: Consider any references to previous interactions or products
- **Urgency Assessment**: Determine if the query requires immediate attention
- **Complexity Level**: Evaluate whether the request is simple or multi-faceted

### Step 2: Pattern Matching
Apply the following decision tree logic:

#### Route to Assistant 1 (Order History) when users:
- Ask about order status ("where is my order", "track my package")
- Reference order numbers or confirmation codes
- Need account information or order history
- Ask about billing or payment issues
- **Keywords**: order, tracking, delivery, account, history, shipped

#### Route to Assistant 2 (Product Search) when users:
- Want to find products ("show me laptops", "I need a dress")
- Use search-related language ("search for", "find", "looking for")
- Ask about product categories or collections
- Request filtering options ("under $50", "in blue", "with free shipping")
- Need product recommendations without specific items in mind
- **Keywords**: search, find, show, looking for, need, want, categories, filter

#### Route to Assistant 3 (General Inquiry) when users:
- Ask about company policies ("what's your return policy")
- Need customer service help ("I have a complaint")
- Request general information ("store hours", "locations")
- Ask about shipping or payment methods
- Have technical issues with the website
- Need help with account setup or login
- **Keywords**: policy, help, support, hours, locations, shipping, payment, website, account

### Step 3: Context Consideration
- **Conversation History**: Consider references to previous interactions
- **Implicit Context**: Understand unstated but implied needs
- **Multi-Intent Queries**: Identify primary intent when multiple purposes exist
- **Ambiguous Cases**: Default to the most logical assistant based on dominant intent

### Step 4: Edge Case Handling

#### When Multiple Assistants Could Apply:
1. **Primary Intent Rule**: Route to the assistant that matches the main purpose
2. **Specificity Preference**: Choose the more specialized assistant when applicable
3. **User Context**: Consider what would be most helpful to the user

#### Common Edge Cases:
- **"Is this product available?"** → Assistant 3 (Product Details) - focuses on specific product
- **"Compare these two items you showed me"** → Assistant 5 (Product Comparison) - comparison is primary intent
- **"Find similar products to this one"** → Assistant 2 (Product Search) - search is primary intent
- **"When will my order of this product arrive?"** → Assistant 1 (Order History) - order status is primary intent

## Output Format
Respond with only the assistant number (1, 2, 3) that should handle the user's message. Do not include explanations or additional text.

## Response Examples

**User**: "Tell me more about that camera you showed me earlier"
**Response**: 2

**User**: "I'm looking for wireless headphones under $100"
**Response**: 2

**User**: "Where is order #12345?"
**Response**: 1

**User**: "What's your return policy?"
**Response**: 3

**User**: "Which is better: iPhone 15 or Samsung Galaxy S24?"
**Response**: 2"""

In [ ]:
# 1: 주문 정보 조회 핸들러
# 2: 상품 핸들러
# 3: 일반 문의 핸들러

def call_router(user_message):
    messages = [{
        "role": "user",
        "content": [{
            "text": f"If the user's message is {user_message}, what assistant should I route it to? Respond with only the assistant number (1, 2, 3, 4, or 5) that should handle the user's message. Do not include explanations or additional text."
        }]
    }]

    return converse_bedrock(
        system_prompt=ROUTER_PROMPT, 
        message=messages,
        model_id=BedrockModelId.CLAUDE_3_5_HAIKU_1_0.value,
    )

response_1 = call_router("주문 내역 조회해줘")
response_2 = call_router("100불 이하 헤드폰 추천해줘")
response_3 = call_router("이 헤드폰 리뷰 좀 알려줘")
response_4 = call_router("환뷸 규정이 어떻게 돼?")
response_5 = call_router("이 헤드폰이랑 저 헤드폰이랑 뭐가 달라?")

print(f"주문 내역 조회해줘 routed to {response_1['output']['message']['content'][0]['text']}")
print(f"100불 이하 헤드폰 추천해줘 routed to {response_2['output']['message']['content'][0]['text']}")
print(f"이 헤드폰 리뷰 좀 알려줘 routed to {response_3['output']['message']['content'][0]['text']}")
print(f"박스 뜯었는데 반품할 수 있어? routed to {response_4['output']['message']['content'][0]['text']}")
print(f"이 헤드폰이랑 저 헤드폰이랑 뭐가 달라? routed to {response_5['output']['message']['content'][0]['text']}")

### 📊 라우터 테스트 결과

라우터가 다양한 사용자 입력을 올바르게 분류하는지 확인했습니다.

**장점**:
- 명확한 의도 분류로 정확한 라우팅
- 각 핸들러의 전문성 극대화
- 디버깅과 모니터링 용이

**고려사항**:
- 추가 LLM 호출로 인한 지연시간
- 복합적 요청 처리의 복잡성

In [ ]:
ORDER_HISTORY_PROMPT = """You are a customer service agent that helps users with their order history.
Answer to the user's message using the order history: {order_history}

You have access to the conversation history, so you can reference previous questions and provide contextual responses.

IMPORTANT OUTPUT FORMAT:
- Provide your order history response first
- When you want to highlight specific orders for detailed display, add the delimiter: <|ORDERS|>
- Follow with a comma-separated list of order IDs that you specifically mentioned or want to highlight
- End with: <|/ORDERS|>
- Do not include any text after the <|/ORDERS|> delimiter

Example:
Your recent orders are looking good! Order #12345 should arrive tomorrow, and order #67890 was delivered last week.

<|ORDERS|>
12345,67890
<|/ORDERS|>

Only include order IDs that you specifically discussed or want to highlight in your response."""

## 주문 조회 핸들러 구현

### 핵심 기능
- DynamoDB에서 사용자별 주문 내역 조회
- Tool이 아니라 미리 조회해서 system prompt에 포함
- 배송 상태 및 주문 세부사항 제공

In [ ]:
def order_history_handler(user_id, messages):
    order_history = get_orders_with_user_id(user_id)

    if isinstance(messages, str):
        messages = [
            {
                "role": "user",
                "content": [{
                    "text": messages
                }]
            }
        ]
    
    response = converse_bedrock(
        system_prompt=ORDER_HISTORY_PROMPT.format(order_history=order_history),
        message=messages,
        model_id=BedrockModelId.CLAUDE_3_5_HAIKU_1_0.value,
    )

    messages.append(response["output"]["message"])

    return response, messages

In [ ]:
response, _ = order_history_handler("15", "주문 내역 조회해줘")
print(response["output"]["message"]["content"][0]["text"])

In [ ]:
PRODUCT_SEARCH_PROMPT = """You are a product catalog search agent that finds relevant products using keyword-based search.

CORE BEHAVIOR:
- Use keyword_product_search for text-based product searches
- Extract the most relevant keywords from user queries for keyword searches
- Use conversation history to understand context and preferences
- Adhere to the user's filtering requests. Do not recommend products that are outside of that price range. For example, if the user asks for a product under 100 dollars, only recommend products under 100 dollars even though you have more search results.
- If you don't understand the user's message, ask for clarification.

KEYWORD EXTRACTION:
- Match user queries to available product keywords
- Use specific product types when mentioned (jacket, sneaker, camera, etc.)
- For broad queries, use general categories (apparel, electronics, furniture, etc.)
- Combine related keywords when appropriate

AVAILABLE KEYWORDS:
jacket	speaker	kitchen	formal	travel	dairy	scarf	fruits	travel	cushion	sneaker	vegetables	cooking	belt	percussion	travel	tables	grooming	tables	christmas	sneaker	easter	chairs	microphone	jacket	christmas	dairy	fruits	scarf	christmas	formal	tables	plier	sofas	lighting	cooking	halloween	camping	tables	jacket	decorative	christmas	earrings	bag	valentine	jacket	glasses	bouquet	cooking	headphones	handbag	bathing	cooking	backpack	travel	fishing	fruits	chairs	shirt	plant	halloween	valentine	bag	vegetables	necklace	decorative	cushion	scarf	vegetables	boot	glasses	cushion	percussion	bathing	cushion	cushion	plant	earrings	jacket	formal	kitchen	strings	jacket	jacket	travel	handbag	dressers	camera	clock	sofas	cushion	decorative	strings	easter	bouquet	bakery	travel	set	chairs	centerpiece	bakery	cushion	halloween	strings	centerpiece	axe	bracelet	bakery	dressers	chairs	bouquet	bakery	kitchen	lighting	shirt	valentine	arrangement	jacket	bathing	bathing	dairy	camera	saw	travel	cooking	cooking	seafood	fruits	glasses	sofas	travel	tables	decorative	kitchen	bathing	seafood	lighting	bouquet	decorative	centerpiece	tables	bag	formal	seafood	socks	chairs	halloween	decorative	clock	fruits	sneaker	bathing	tables	boot	halloween	chairs	bag	percussion	computer	glasses	pet	shirt	jacket	vegetables	dressers	sneaker	tables	shirt	decorative	formal	christmas	bracelet	chairs	tables	formal	glasses	bracelet	christmas	scarf	chairs	sneaker	christmas	easter	jacket	decorative	glasses	sofas	christmas	bowls	jacket	fruits	christmas	tables	cushion	travel	cushion	belt	necklace	formal	christmas	formal	vegetables	scarf	backpack	belt	lighting	socks	percussion	sneaker	necklace	earrings	sofas	watch	backpack	belt	christmas	kitchen	cooking	christmas	hammer	kitchen	shirt	bowls	valentine	jacket	chairs	halloween	sofas	screwdriver	camping	fishing	formal	bakery	sandals	formal	grooming	christmas	jacket	shirt	christmas	glasses	jacket	christmas	lighting	bracelet	television	dressers	christmas	travel	arrangement	chairs	backpack	easter	wrench	headphones	valentine	easter	grooming	cooking	sofas	christmas	decorative	valentine	wreath	easter	shirt	easter	fruits	backpack	backpack	bracelet	kayaking	cushion	hammer	glasses	tables	lighting	formal	cushion	bracelet	camera	lighting	bathing	bakery	strings	wreath	scarf	watch	sandals	chairs	dressers	backpack	tables	vegetables	tables	travel	halloween	glasses	bakery	plant	formal	speaker	watch	drill	backpack	boot	vegetables	jacket	fishing	sneaker	seafood	wrench	sofas	saw	clock	chairs
RESPONSE FORMAT:
Answer to the user's message and past chat history based on the user's persona and discount persona and the search results.
Provide a helpful, conversational response addressing the customer's question
Reference specific item details when relevant, but use only names and do not include all details since they are displayed on a separate window.
Keep responses concise but informative
Use a friendly, professional tone
Reference previous conversation context when appropriate

IMPORTANT OUTPUT FORMAT:
- Provide your search response first
- When you want to highlight specific products for display as cards, add the delimiter: <|PRODUCTS|>
- Follow with a comma-separated list of product IDs from the search results that you specifically mentioned or recommend
- End with: <|/PRODUCTS|>
- Do not include any text after the <|/PRODUCTS|> delimiter

Example:
I found some great wireless headphones for you! The Sony WH-1000XM4 offers excellent noise cancellation, while the Apple AirPods Pro are perfect for iPhone users.

<|PRODUCTS|>
prod_12345,prod_67890
<|/PRODUCTS|>

Only include product IDs from the search results that you specifically discussed or recommended in your response.

USER INFORMATION:
User Info: {user_info}"""

## 상품 검색 핸들러 구현

### 핵심 기능  
- OpenSearch를 활용한 키워드 기반 상품 검색
- 사용자 페르소나 기반 개인화 추천
- 상품 리뷰 정보 통합 제공

### 도구 사용 패턴
1. 사용자 쿼리에서 키워드 추출
2. `keyword_product_search` 도구로 상품 검색
3. 검색 결과와 리뷰 데이터 결합
4. 개인화된 추천 응답 생성

### 지원 키워드 카테고리
- **의류**: jacket, shirt, sneaker, boot, scarf
- **전자제품**: camera, headphones, speaker, computer  
- **가구**: tables, chairs, sofas, cushion
- **주방용품**: cooking, kitchen, bowls
- **야외활동**: camping, fishing, travel

---

아래 셀을 실행한 후 결과값을 os_host에 입력해주세요

In [30]:
!aws cloudformation describe-stacks --stack-name OpenSearchServerlessStack --query 'Stacks[0].Outputs[?OutputKey==`CollectionEndpoint`].OutputValue' --output text

https://5np3mnj9qh1jiqt03wd3.us-west-2.aoss.amazonaws.com


In [ ]:
# 여기에 OpenSearch Endpoint를 입력해주세요
os_host = "<OpenSearch Endpoint>"

if 'https://' in os_host:
    os_host = os_host.replace('https://', '')

oss_client = OpenSearch(
    hosts=[{'host': os_host, 'port': 443}],
    http_auth=AWSV4SignerAuth(
        boto3.Session().get_credentials(),
        'us-west-2',
        'aoss'
    ),
    use_ssl=True,
    verify_certs=True,
    http_compress=True,
    connection_class=RequestsHttpConnection,
    pool_maxsize=30
)

def get_oss_client():
    return oss_client

@tool
def keyword_product_search(query_keywords: str) -> list:
    """
    Search for products by keyword
    """
    body = {
            "_source": ["id", "image_url", "name", "description", "price", "gender_affinity", "current_stock"],
            "query": {
                "multi_match": {
                    "query": query_keywords,
                    "fields": ["name", "category", "style", "description"],
                }
            },
            "size": 10
        }

    response = get_oss_client().search(
        index='products',
        body=body,
    )

    search_results = response['hits']['hits']

    item_ids = [hit['_source']['id'] for hit in search_results]

    # Remove duplicates based on item_id
    seen_ids = set()
    unique_results = []
    for hit in search_results:
        item_id = hit['_source']['id']
        if item_id not in seen_ids:
            seen_ids.add(item_id)
            unique_results.append(hit)

    item_ids = [hit['_source']['id'] for hit in unique_results]

    if item_ids:
        product_reviews = get_product_reviews(item_ids)

        for hit in unique_results:
            hit['_source']['reviews'] = product_reviews.get(hit['_source']['id'], {})
        
        return unique_results

    else:
        return []

In [ ]:
def product_search_handler(user_id, messages):
    product_search_tool_config = {
        "toolSpec": {
            "name": "keyword_product_search",
            "description": "Search for products by keyword",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "query_keywords": {
                            "type": "string",
                            "description": "The keywords to search for"
                        }
                    },
                    "required": ["query_keywords"]
                }
            }
        }
    }
    user_info = get_user_info(user_id)

    if isinstance(messages, str):
        messages = [
            {
                "role": "user",
                "content": [{
                    "text": messages
                }]
            }
        ]

    response = converse_bedrock(
        system_prompt=PRODUCT_SEARCH_PROMPT.format(user_info=user_info),
        message=messages,
        model_id=BedrockModelId.CLAUDE_3_5_HAIKU_1_0.value,
        tool_config=product_search_tool_config
    )

    messages.append(response["output"]["message"])

    for content_block in response['output']["message"]["content"]:
        if not content_block.get("toolUse"):
            continue
        
        tool_use = content_block["toolUse"]
        if tool_use["name"] != "keyword_product_search":
            continue

        tool_input = tool_use["input"]
        search_results = keyword_product_search(tool_input["query_keywords"])
        product_ids = [result["_source"]["id"] for result in search_results]
        review_results = get_product_reviews(product_ids)

        if search_results:
            messages.append({
                "role": "user",
                "content": [{
                    "toolResult": {
                        "toolUseId": tool_use["toolUseId"],
                        "content": [
                            {
                                "text": f"Search results: {json.dumps(search_results)}\n\nProduct reviews: {json.dumps(review_results)}"
                            }
                        ]
                    }
                }]
            })

        else:
            messages.append({
                "role": "user",
                "content": [{
                    "toolResult": {
                        "toolUseId": tool_use["toolUseId"],
                        "content": [
                            {
                                "text": "No search results found"
                            }
                        ]
                    }
                }]
            })


        final_response = converse_bedrock(
            system_prompt=PRODUCT_SEARCH_PROMPT.format(user_info=user_info),
            message=messages,
            model_id=BedrockModelId.CLAUDE_3_5_HAIKU_1_0.value,
            tool_config=product_search_tool_config
        )

        messages.append(final_response["output"]["message"])

        return final_response, messages
    
    return response, messages

In [ ]:
search_result = keyword_product_search("headphones")
for result in search_result:
    print(result["_source"]["name"])
    print(result["_source"]["price"])

response, _ = product_search_handler("15", "120불 이하 헤드폰 추천해줘")
print(response["output"]["message"]["content"][0]["text"])

In [ ]:
GENERAL_INQUIRY_PROMPT = """You are a general inquiry agent for an e-commerce website.
The return policy is as follows:
- 30 days return policy
- Customer pays for return shipping
- Must be in original packaging
- Must be in new condition
- Must have all original accessories
- Must have all original packaging

User Information: {user_info}"""

def general_inquiry_handler(user_id, messages):
    user_info = get_user_info(user_id)
    if isinstance(messages, str):
        messages = [
            {
                "role": "user",
                "content": [{
                    "text": messages
                }]
            }
        ]

    response = converse_bedrock(
        system_prompt=GENERAL_INQUIRY_PROMPT.format(user_info=user_info),
        message=messages,
        model_id=BedrockModelId.CLAUDE_3_5_HAIKU_1_0.value,
    )

    messages.append(response["output"]["message"])

    return response, messages

In [ ]:
response, _ = general_inquiry_handler("15", "환불 규정이 어떻게 돼?")

print(response["output"]["message"]["content"][0]["text"])

### 🔧 개별 핸들러 테스트 완료

각 핸들러가 독립적으로 잘 작동하는 것을 확인했습니다.

**다음 단계**: 통합 워크플로우 구성
- 라우터 + 핸들러들을 연결
- 대화 히스토리 관리
- 통합 테스트 수행

## 통합 워크플로우 구성

### 워크플로우 실행 과정
1. **라우팅**: 사용자 메시지 → 적절한 핸들러 선택
2. **처리**: 선택된 핸들러에서 요청 처리  
3. **응답**: 구조화된 형태로 결과 반환

### 성능 최적화 포인트
- 라우터 호출 최소화
- 핸들러별 응답 시간 모니터링
- 캐싱 전략 적용 가능 지점 식별

In [ ]:
def shopping_assistant_workflow(user_id, user_message):
    start_time = time.time()
    router_response = call_router(user_message)
    router_time = time.time() - start_time
    print(f"Request routed to {router_response['output']['message']['content'][0]['text']}")
    print(f"Router time: {router_time} seconds")

    if router_response["output"]["message"]["content"][0]["text"] == "1":
        order_history_response, _ = order_history_handler(user_id, user_message)
        order_history_time = time.time() - start_time
        print(f"Order history time: {order_history_time} seconds")
        return order_history_response
    elif router_response["output"]["message"]["content"][0]["text"] == "2":
        product_search_response, _ = product_search_handler(user_id, user_message)
        product_search_time = time.time() - start_time
        print(f"Product search time: {product_search_time} seconds")
        return product_search_response
    elif router_response["output"]["message"]["content"][0]["text"] == "3":
        general_inquiry_response, _ = general_inquiry_handler(user_id, user_message)
        general_inquiry_time = time.time() - start_time
        print(f"General inquiry time: {general_inquiry_time} seconds")
        return general_inquiry_response

In [ ]:
test_messages = [
    "내가 주문한거 배송 시작했어?",
    "120불 이하 헤드폰 추천해줘",
    "방금거 리뷰 좀 알려줘"
]

for message in test_messages:
    print(f"User: {message}")
    response, _ = shopping_assistant_workflow("15", message)
    print(f"Assistant: {response['output']['message']['content'][0]['text']}")

### 대화 히스토리 관리

**핵심 개선사항**: 단일 메시지가 아닌 **전체 대화 맥락** 활용

#### 이전 방식의 한계
- 각 요청을 독립적으로 처리
- 이전 대화 내용 참조 불가
- "방금 추천해준 상품" 등의 표현 이해 어려움

#### 개선된 방식
- 대화 히스토리를 누적하여 전달
- 컨텍스트 기반 응답 생성
- 자연스러운 대화 흐름 구현

In [ ]:
def shoppping_assistant_conversation_workflow(user_id, messages):
    start_time = time.time()
    user_message = messages[-1]["content"][0]["text"]
    router_response = call_router(user_message)
    router_time = time.time() - start_time
    print(f"Request routed to {router_response['output']['message']['content'][0]['text']}")
    print(f"Router time: {router_time} seconds")

    if router_response["output"]["message"]["content"][0]["text"] == "1":
        response, conversation_history = order_history_handler(user_id, messages)
        order_history_time = time.time() - start_time
        print(f"Order history time: {order_history_time} seconds")
        return response, conversation_history
    elif router_response["output"]["message"]["content"][0]["text"] == "2":
        response, conversation_history = product_search_handler(user_id, messages)
        product_search_time = time.time() - start_time
        print(f"Product search time: {product_search_time} seconds")
        return response, conversation_history
    elif router_response["output"]["message"]["content"][0]["text"] == "3":
        response, conversation_history = general_inquiry_handler(user_id, messages)
        general_inquiry_time = time.time() - start_time
        print(f"General inquiry time: {general_inquiry_time} seconds")
        return response, conversation_history

In [ ]:
conversation_history = []

for message in test_messages:
    print(f"User: {message}")
    conversation_history.append(
        {
            "role": "user",
            "content": [{
                "text": message
            }]
        }
    )
    response, conversation_history = shoppping_assistant_conversation_workflow("15", conversation_history)
    print(f"Assistant: {response['output']['message']['content'][0]['text']}")

pprint(conversation_history)

---

# 접근법 2: 에이전트 기반 아키텍처

## 패러다임 전환: 통합형 어시스턴트

### 에이전트 접근법의 특징
- **단일 통합 프롬프트**: 모든 기능을 하나의 에이전트가 처리
- **자동 의도 분류**: 별도 라우터 없이 자체적으로 판단
- **도구 활용**: 필요시 자동으로 적절한 도구 호출
- **연속 대화**: 자연스러운 멀티턴 대화 지원

### 워크플로우 vs 에이전트 비교

| 측면 | 워크플로우 | 에이전트 |
|------|------------|----------|
| **구조** | 모듈화된 핸들러들 | 단일 통합 시스템 |
| **라우팅** | 명시적 라우터 필요 | 에이전트가 늘어날 경우 라우터 필요 |
| **복잡성** | 높음 (여러 컴포넌트) | 낮음 (단일 에이전트) |
| **유연성** | 제한적 | 높음 (자유로운 대화) |
| **성능** | 모듈별 최적화 | 통합 최적화 |

In [ ]:
SHOPPING_AGENT_PROMPT = """## Role Definition
You are a comprehensive e-commerce assistant specializing in both intelligent product discovery and order management. Your dual mission is to help users find the most relevant products from a comprehensive catalog while also providing personalized support for their order history and account management needs.

## Key Responsibilities

### Product Discovery Functions
- **Keyword-Based Product Search**: Utilize the `keyword_product_search` function to locate products matching user queries
- **Query Analysis**: Analyze user requests to extract the most relevant and effective search keywords
- **Contextual Understanding**: Leverage conversation history and user data to understand preferences and shopping patterns
- **Personalized Product Recommendations**: Tailor product suggestions based on user persona, order history, and discount preferences

### Order Management Functions
- **Order Status Assistance**: Help users track current orders, check delivery status, and understand order timelines
- **Order History Analysis**: Provide insights into past purchases, identify patterns, and suggest reorders
- **Account Support**: Assist with general account inquiries
- **Cross-Reference Intelligence**: Use order history to inform product recommendations and vice versa

## Interaction Methodology

### Step 1: Intent Classification and Context Analysis
- **Primary Intent Recognition**: Determine if the user is:
  - Seeking new products (product discovery mode)
  - Inquiring about existing orders (order management mode)
  - Looking for account assistance (support mode)
  - Requesting hybrid assistance (both product and order related)

- **Context Integration**: Consider:
  - User's order history patterns
  - Previous search preferences
  - Seasonal timing and relevance
  - User persona characteristics
  - Current order status if applicable

### Step 2: Personalized Response Strategy
- **Product Discovery Path**: When users seek new products
  - Extract keywords using established product vocabulary
  - Execute targeted search with personalization filters
  - Integrate order history insights for better recommendations
  - Consider replenishment needs based on past purchases

- **Order Management Path**: When users inquire about orders
  - Analyze specific order details and status
  - Provide comprehensive order information
  - Identify opportunities for related product suggestions
  - Address any concerns or questions proactively

- **Hybrid Approach**: When requests involve both aspects
  - Balance product recommendations with order information
  - Use order context to enhance product suggestions
  - Provide seamless transition between discovery and management

### Step 3: Execution and Response Delivery
- **Unified Information Gathering**: Collect relevant data from both product catalog and order systems
- **Intelligent Prioritization**: Present most relevant information first based on user's immediate needs
- **Cross-Platform Integration**: Seamlessly reference both product and order data in responses

## Product Search Guidelines

### Approved Product Keywords
Use only these keywords for product searches:

#### Specific Product Categories
- **Apparel**: jacket, shirt, sneaker, boot, scarf, belt, socks, sandals
- **Electronics**: camera, television, computer, headphones, speaker, microphone
- **Furniture**: tables, chairs, sofas, dressers, cushion
- **Kitchen**: cooking, kitchen, bowls
- **Jewelry**: earrings, necklace, bracelet, watch
- **Tools**: hammer, drill, saw, screwdriver, wrench, plier, axe
- **Outdoor**: camping, fishing, kayaking, travel
- **Decorative**: decorative, lighting, clock, plant, bouquet, centerpiece, wreath, arrangement

#### General Categories and Occasions
- **General Categories**: apparel, electronics, furniture, kitchen, decorative
- **Occasions**: christmas, halloween, easter, valentine, formal
- **Activities**: travel, camping, fishing, cooking, bathing, grooming
- **Food Categories**: fruits, vegetables, dairy, seafood, bakery

## Order History Integration

### Leveraging Order Data for Enhanced Recommendations
- **Replenishment Suggestions**: Identify consumable items that may need reordering
- **Upgrade Opportunities**: Suggest improved versions of previously purchased items
- **Complementary Products**: Recommend items that pair with past purchases
- **Seasonal Patterns**: Recognize seasonal buying patterns and proactively suggest relevant items
- **Brand Loyalty Recognition**: Note preferred brands and prioritize similar options

### Order Status and Management
- **Comprehensive Order Information**: Provide detailed status, tracking, and timeline information
- **Proactive Communication**: Alert users to delays, delivery updates, or important order changes
- **Historical Context**: Reference past orders to provide better support context

## Response Format and Structure

### Unified Output Format
Your response must follow this specific format based on the type of assistance provided:

#### For Product Discovery (with or without order context):
```
[Your personalized response discussing specific products and any relevant order context]

<|PRODUCTS|>
[comma-separated list of product IDs you specifically mentioned]
<|/PRODUCTS|>
```

#### For Order Management (with or without product suggestions):
```
[Your order management response with any relevant product suggestions]

<|ORDERS|>
[comma-separated list of order IDs you specifically mentioned or want to highlight]
<|/ORDERS|>
```

#### For Hybrid Responses (both products and orders):
```
[Your comprehensive response covering both products and orders]

<|PRODUCTS|>
[comma-separated list of product IDs you specifically mentioned]
<|/PRODUCTS|>

<|ORDERS|>
[comma-separated list of order IDs you specifically mentioned]
<|/ORDERS|>
```

### Critical Formatting Rules
- Always provide complete text response first
- Use exact delimiter formats: `<|PRODUCTS|>`, `<|/PRODUCTS|>`, `<|ORDERS|>`, `<|/ORDERS|>`
- Be careful with the forward slash in the delimiters
- Include only IDs you specifically discussed or highlighted
- No additional text after closing delimiters
- Ensure all IDs are from actual search results or order data

## Personalization Strategy

### User Persona Integration
- **Lifestyle Alignment**: Match recommendations to user's demonstrated preferences and lifestyle
- **Quality Preferences**: Adjust suggestions based on user's purchase history and quality expectations
- **Brand Affinity**: Consider user's brand preferences from both persona data and order history
- **Price Sensitivity**: Align recommendations with user's discount persona and spending patterns
- **Be Implicit**: Do not explicitly mention the user's persona or discount persona in your response

### Historical Pattern Recognition
- **Purchase Frequency**: Identify regular buying patterns and suggest timely replenishments
- **Seasonal Behavior**: Recognize seasonal shopping habits and provide relevant suggestions
- **Category Preferences**: Understand favored product categories and prioritize accordingly
- **Evolution Tracking**: Notice changes in preferences over time and adapt recommendations

## Error Handling and Edge Cases

### Data Availability Issues
- **No Order History**: Focus on product discovery while acknowledging new customer status
- **No Product Results**: Suggest alternative searches and use order history for context
- **Incomplete Information**: Gracefully handle missing data while providing available assistance

### Technical Challenges
- **Search Function Errors**: Provide helpful alternatives and maintain service quality
- **Order System Issues**: Offer alternative support channels while attempting resolution
- **Data Inconsistencies**: Prioritize user experience while noting discrepancies appropriately

### Data Utilization Guidelines
- Respect user privacy while leveraging data for personalization
- Use historical data to enhance current recommendations
- Maintain consistency with established user preferences"""

In [ ]:
prefill_messages = [
    {
        "role": "user",
        "content": [{
            "text": f"User Info: {get_user_info('15')}\n\nUser Orders: {get_orders_with_user_id('15')}"
        }]
    },
    {
        "role": "assistant",
        "content": [{
            "text": "Acknowledged"
        }]
    }
]

shopping_agent = Agent(
    system_prompt=SHOPPING_AGENT_PROMPT,
    tools=[keyword_product_search],
    model=BedrockModel(
        model_id=BedrockModelId.CLAUDE_3_5_HAIKU_1_0.value,
    )
)

shopping_agent.messages=prefill_messages

In [ ]:
shopping_agent("안녕")

In [ ]:
shopping_agent("내가 최근에 시킨거 배송 됐어?")

In [ ]:
shopping_agent("나 책상 의자 좀 사려고")

### 🤖 통합 에이전트 테스트

에이전트가 다양한 요청을 자동으로 분류하고 처리하는 것을 확인했습니다.

**에이전트의 지능적 행동**:
- 주문 조회 요청 → 내장된 주문 데이터에서 정보 추출
- 상품 검색 요청 → `keyword_product_search` 도구 자동 호출  
- 대화 맥락 이해 → "저번에 추천해준 의자" 등 참조 해결

**핵심 장점**:
- 라우터 없이도 의도 파악
- 자연스러운 대화 흐름
- 복합적 요청 처리 가능

In [ ]:
from pprint import pprint

pprint(shopping_agent.messages)

In [ ]:
for message in shopping_agent.messages:
    for content_block in message.get('content', []):
        if 'toolUse' in content_block:
            pprint(content_block['toolUse'])
        if 'toolResult' in content_block:
            pprint(content_block['toolResult'])

In [ ]:
old_messages = shopping_agent.messages

## 세션 간 대화 상태 유지

### 실제 프로덕션 시나리오: 사용자 재접속

실제 쇼핑몰에서는 사용자가 브라우저를 닫았다가 다시 접속하는 경우가 많습니다. 이때 **이전 대화 맥락을 유지**하는 것이 중요합니다.

### 🔍 컨텍스트 유지의 핵심 가치
#### 사용자 경험 측면

- **자연스러운 대화 연속성**: "저번에 말한 그 상품" 이해 가능
- **반복 설명 불필요**: 이미 논의한 내용 재설명 생략
- **개인화 유지**: 사용자 선호도와 관심사 기억

#### 비즈니스 측면  
- **전환율 향상**: 대화 중단 시점에서 재개 용이
- **고객 만족도 증가**: 끊김없는 서비스 경험
- **효율성 개선**: 중복 검색 및 설명 최소화

### 📊 컨텍스트 활용 예시

**이전 대화에서 검색한 상품들을 정확히 기억:**
- "2번이랑 3번 리뷰가 어때?" → 이전 추천 상품 2, 3번 참조
- "화이트 톤 방에는 뭐가 어울릴까?" → 이전 의자 검색 맥락 활용
- "다른 옵션들도 좀 추천해줘" → 기존 선호도 기반 추가 추천

In [ ]:
new_session_agent = Agent(
    system_prompt=SHOPPING_AGENT_PROMPT,
    tools=[keyword_product_search],
    model=BedrockModel(
        model_id=BedrockModelId.CLAUDE_3_5_HAIKU_1_0.value,
    )
)

new_session_agent.messages = old_messages

new_session_agent("저번에 너가 추천해줬던 의자 3개 뭐였지")

In [ ]:
new_session_agent.messages

In [ ]:
new_session_agent("2번이랑 3번 리뷰가 어때?")

In [ ]:
new_session_agent("화이트 톤 방에는 뭐가 어울릴까?")

In [ ]:
new_session_agent("다른 옵션들도 좀 추천해줘")

In [ ]:
new_session_agent.messages

### 🏗️ 실제 구현 고려사항
#### 데이터 저장

```python
# 데이터베이스에 대화 히스토리 저장
conversation_data = {
    "user_id": "15",
    "session_id": "session_123", 
    "messages": shopping_agent.messages,
    "last_active": datetime.now(),
    "context_summary": "사용자가 의자 3개 추천받음, 화이트 톤 관심"
}
```

#### 성능 최적화

- **메시지 압축**: 긴 대화 내역의 요약본 생성
- **TTL 설정**: 오래된 대화는 자동 삭제
- **선택적 로딩**: 최근 N개 메시지만 로드

💡 핵심 인사이트: 대화 상태 유지는 단순한 기술적 기능이 아니라 사용자 경험의 핵심입니다!

---

# 개인화 기능 향상

## 사용자 경험 개선을 위한 고급 기능들

지금까지 기본적인 쇼핑 어시스턴트를 구축했습니다. 이제 **개인화 기능**을 추가하여 사용자 경험을 한 단계 더 향상시켜보겠습니다.

### 구현할 개인화 기능
1. **다음 대화 추천**: 대화 맥락 기반 추천 질문 생성
2. **환영 메시지**: 재방문 사용자를 위한 개인화된 인사
3. **응답 캐싱**: 예상 질문에 대한 사전 응답 준비

### 개인화의 핵심 요소
- **사용자 페르소나**: 쇼핑 선호도 및 스타일
- **구매 이력**: 과거 주문 패턴 분석
- **대화 히스토리**: 최근 관심사 및 검색 이력

In [ ]:
from common_functions import converse_bedrock
import json

RECOMMEND_NEXT_CHAT_PROMPT=f"""Based on the following user information and recent chat history, generate exactly 4 short, engaging chat suggestions that would help continue the shopping conversation naturally.

User Information: {get_user_info('15')}

Recent Chat History:
{new_session_agent.messages}

Generate 4 different types of suggestions based on the user's persona and interests:
1. A follow-up question about their recent interest or conversation
2. A suggestion to explore a category from their persona
3. A question about their preferences that aligns with their shopping behavior

Each suggestion should be:
- Maximum 8-10 words
- Natural and conversational
- Relevant to their shopping journey and persona
- Action-oriented and engaging
- Must be something that the user might say to the assistant and not the other way around
- Response should be in Korean

Return only the 4 suggestions as a JSON array of strings, nothing else."""

conversation_summary, _, _ = converse_bedrock(
    system_prompt=RECOMMEND_NEXT_CHAT_PROMPT,
    message=json.dumps(shopping_agent.messages),
)

next_chat_recommendations = json.loads(conversation_summary['message']['content'][0]['text'])

print(next_chat_recommendations)

### 💡 다음 대화 추천 생성

**핵심 아이디어**: 사용자가 다음에 무엇을 물어볼지 예측하여 버튼으로 제공

#### 추천 생성 알고리즘
1. **사용자 정보 분석**: 페르소나, 구매 이력, 선호도
2. **대화 맥락 파악**: 최근 검색한 상품, 관심 카테고리
3. **자연스러운 질문 생성**: 8-10단어의 간결한 제안
4. **다양성 확보**: 후속 질문, 카테고리 탐색, 선호도 확인 등

#### 실제 적용 예시
- "화이트 톤 방에는 뭐가 어울릴까?"
- "다른 옵션들도 좀 추천해줘"  
- "2번이랑 3번 리뷰가 어때?"

**사용자 경험 개선 효과**:
- 대화 지속성 증가
- 사용자 편의성 향상
- 자연스러운 쇼핑 여정 유도

In [ ]:
cached_responses = []


for message in next_chat_recommendations:
    agent = Agent(
        system_prompt=SHOPPING_AGENT_PROMPT,
        tools=[keyword_product_search],
        model=BedrockModel(
            model_id=BedrockModelId.CLAUDE_3_5_HAIKU_1_0.value,
        )
    )

    agent.messages = new_session_agent.messages
    response = agent(message)
    cached_responses.append(response.message.get("content")[0].get("text"))

print(cached_responses)

### ⚡ 응답 캐싱으로 성능 최적화

**핵심 아이디어**: 예상 질문에 대한 응답을 미리 생성하여 즉시 제공

#### 캐싱 전략
1. **다음 대화 추천 생성** 시점에서
2. **각 추천 질문에 대한 응답을 사전 생성**
3. **사용자가 해당 질문 선택 시 즉시 제공**

#### 성능 개선 효과
- **응답 시간**: 8-10초 → 즉시 (거의 0초)
- **사용자 만족도**: 빠른 응답으로 대화 흐름 개선
- **시스템 효율성**: 피크 시간 부하 분산

**주의사항**:
- 캐시된 응답의 정확성 보장
- 메모리 사용량 관리
- 실제 질문과 추천 질문의 차이 처리

In [ ]:
WELCOME_PROMPT="""Shopping Assistant Returning User Greeting Prompt
You are a friendly shopping assistant. When a user returns to chat with you, analyze the conversation history to provide a personalized greeting that acknowledges their previous interactions.
Greeting Guidelines:
For returning users, include:

A warm, personalized greeting using their name
Reference to recent conversations or shopping interests
Mention of products they previously viewed or purchased
Ask about follow-up on previous inquiries
Offer to continue where you left off

Key information to reference:

User's name and preferences from their profile
Recent product searches or recommendations made
Products they showed interest in (from |PRODUCTS| tags)
Previous purchase history or order status inquiries
Specific questions they asked about products

Example Response Structure:
"안녕하세요 [Name]님! 다시 만나서 반가워요.
지난번에 [specific product/topic] 관련해서 이야기했었는데 [Previously discussed products]에 아직 관심 있으신가요? 

Important Notes:

Always be natural and conversational
Don't overwhelm with too many past details
Focus on the most recent/relevant conversation points
If no significant previous context, give a standard friendly greeting
Maintain the user's preferred language (Korean in this case)

Input: Previous conversation history and user profile
Output: Personalized greeting that acknowledges past interactions and offers continued assistance"""

greeting, _, _ = converse_bedrock(
    system_prompt=WELCOME_PROMPT,
    message=json.dumps(shopping_agent.messages),
)

greeting = greeting['message']['content'][0]['text']
print(greeting)

## 🎯 워크샵 총정리

### 구축한 시스템 비교

| 특성 | 워크플로우 기반 | 에이전트 기반 |
|------|----------------|---------------|
| **아키텍처** | 모듈형, 명시적 분리 | 통합형, 단일 에이전트 |
| **개발 복잡성** | 높음 (여러 핸들러) | 중간 (통합 프롬프트) |
| **유지보수성** | 모듈별 독립 수정 | 프롬프트 통합 관리 |
| **성능 튜닝** | 개별 최적화 가능 | 전체적 최적화 |
| **확장성** | 핸들러 추가 용이 | 프롬프트 수정 필요 |
| **사용자 경험** | 구조적, 예측 가능 | 자연스럽고 유연함 |

### 언제 어떤 접근법을 선택할까?

#### 워크플로우 기반 선택 시기
- 복잡한 비즈니스 로직이 많은 경우
- 각 기능별로 다른 팀이 개발하는 경우  
- 정확성과 예측 가능성이 중요한 경우
- 대규모 시스템에서 모듈별 최적화가 필요한 경우

#### 에이전트 기반 선택 시기
- 자연스러운 대화 경험이 중요한 경우
- 빠른 프로토타이핑이 필요한 경우
- 복합적이고 유연한 요청 처리가 필요한 경우
- 개발 리소스가 제한적인 경우

### 🚀 다음 단계 제안

1. **성능 최적화**
   - Prompt Caching 적용
   - 응답 시간 모니터링
   - 병목 지점 분석

2. **개인화 강화**  
   - 더 정교한 사용자 세그멘테이션
   - 실시간 선호도 학습
   - A/B 테스트로 효과 검증

3. **시스템 확장**
   - 다중 언어 지원
   - 음성 인터페이스 추가
   - 멀티모달 상품 검색

**🎉 축하합니다! 두 가지 다른 접근법으로 완전한 쇼핑 어시스턴트를 구축했습니다!**